# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 15 2022 8:32AM,239611,18,INNOVA-8147396,Innova Softgel LLC,Released
1,Jun 15 2022 8:30AM,239612,19,ACG-2102490333,ACG North America LLC,Released
2,Jun 15 2022 8:19AM,239610,10,MET211213,"Methapharm, Inc.",Released
3,Jun 15 2022 8:05AM,239609,19,ADV80003345,"AdvaGen Pharma, Ltd",Released
4,Jun 15 2022 8:05AM,239609,19,ADV80003346,"AdvaGen Pharma, Ltd",Released
5,Jun 15 2022 8:05AM,239609,19,ADV80003347,"AdvaGen Pharma, Ltd",Released
6,Jun 15 2022 8:05AM,239609,19,ADV80003348,"AdvaGen Pharma, Ltd",Released
7,Jun 15 2022 8:05AM,239609,19,ADV80003349,"AdvaGen Pharma, Ltd",Released
8,Jun 15 2022 8:05AM,239609,19,ADV80003350,"AdvaGen Pharma, Ltd",Released
9,Jun 15 2022 8:05AM,239609,19,ADV80003351,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,239608,Released,15
44,239609,Released,9
45,239610,Released,1
46,239611,Released,1
47,239612,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239608,NaN,NaN,15.0
239609,NaN,NaN,9.0
239610,NaN,NaN,1.0
239611,NaN,NaN,1.0
239612,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,74.0,0.0,79.0
239526,9.0,2.0,3.0
239533,20.0,2.0,0.0
239535,11.0,3.0,16.0
239544,0.0,2.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,74,0,79
239526,9,2,3
239533,20,2,0
239535,11,3,16
239544,0,2,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,74,0,79
1,239526,9,2,3
2,239533,20,2,0
3,239535,11,3,16
4,239544,0,2,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,74,,79
1,239526,9,2,3
2,239533,20,2,
3,239535,11,3,16
4,239544,,2,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 15 2022 8:32AM,239611,18,Innova Softgel LLC
1,Jun 15 2022 8:30AM,239612,19,ACG North America LLC
2,Jun 15 2022 8:19AM,239610,10,"Methapharm, Inc."
3,Jun 15 2022 8:05AM,239609,19,"AdvaGen Pharma, Ltd"
12,Jun 15 2022 8:00AM,239608,19,"AdvaGen Pharma, Ltd"
27,Jun 15 2022 7:58AM,239607,12,Hush Hush
41,Jun 15 2022 7:47AM,239606,22,"NBTY Global, Inc."
42,Jun 15 2022 12:00AM,239603,10,"Methapharm, Inc."
46,Jun 15 2022 12:00AM,239604,10,"Methapharm, Inc."
68,Jun 15 2022 12:00AM,239601,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 15 2022 8:32AM,239611,18,Innova Softgel LLC,,,1
1,Jun 15 2022 8:30AM,239612,19,ACG North America LLC,,,1
2,Jun 15 2022 8:19AM,239610,10,"Methapharm, Inc.",,,1
3,Jun 15 2022 8:05AM,239609,19,"AdvaGen Pharma, Ltd",,,9
4,Jun 15 2022 8:00AM,239608,19,"AdvaGen Pharma, Ltd",,,15
5,Jun 15 2022 7:58AM,239607,12,Hush Hush,,,14
6,Jun 15 2022 7:47AM,239606,22,"NBTY Global, Inc.",,1,
7,Jun 15 2022 12:00AM,239603,10,"Methapharm, Inc.",,,22
8,Jun 15 2022 12:00AM,239604,10,"Methapharm, Inc.",,,4
9,Jun 15 2022 12:00AM,239601,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 8:32AM,239611,18,Innova Softgel LLC,1,,
1,Jun 15 2022 8:30AM,239612,19,ACG North America LLC,1,,
2,Jun 15 2022 8:19AM,239610,10,"Methapharm, Inc.",1,,
3,Jun 15 2022 8:05AM,239609,19,"AdvaGen Pharma, Ltd",9,,
4,Jun 15 2022 8:00AM,239608,19,"AdvaGen Pharma, Ltd",15,,
5,Jun 15 2022 7:58AM,239607,12,Hush Hush,14,,
6,Jun 15 2022 7:47AM,239606,22,"NBTY Global, Inc.",,1,
7,Jun 15 2022 12:00AM,239603,10,"Methapharm, Inc.",22,,
8,Jun 15 2022 12:00AM,239604,10,"Methapharm, Inc.",4,,
9,Jun 15 2022 12:00AM,239601,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 8:32AM,239611,18,Innova Softgel LLC,1,,
1,Jun 15 2022 8:30AM,239612,19,ACG North America LLC,1,,
2,Jun 15 2022 8:19AM,239610,10,"Methapharm, Inc.",1,,
3,Jun 15 2022 8:05AM,239609,19,"AdvaGen Pharma, Ltd",9,,
4,Jun 15 2022 8:00AM,239608,19,"AdvaGen Pharma, Ltd",15,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 8:32AM,239611,18,Innova Softgel LLC,1.0,NaN,NaN
1,Jun 15 2022 8:30AM,239612,19,ACG North America LLC,1.0,NaN,NaN
2,Jun 15 2022 8:19AM,239610,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Jun 15 2022 8:05AM,239609,19,"AdvaGen Pharma, Ltd",9.0,NaN,NaN
4,Jun 15 2022 8:00AM,239608,19,"AdvaGen Pharma, Ltd",15.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 8:32AM,239611,18,Innova Softgel LLC,1.0,0.0,0.0
1,Jun 15 2022 8:30AM,239612,19,ACG North America LLC,1.0,0.0,0.0
2,Jun 15 2022 8:19AM,239610,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Jun 15 2022 8:05AM,239609,19,"AdvaGen Pharma, Ltd",9.0,0.0,0.0
4,Jun 15 2022 8:00AM,239608,19,"AdvaGen Pharma, Ltd",15.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3354101,42.0,5.0,37.0
12,479151,14.0,2.0,0.0
15,239446,79.0,0.0,74.0
18,1437485,6.0,0.0,0.0
19,2156300,44.0,2.0,0.0
20,479086,18.0,28.0,12.0
21,718777,3.0,0.0,0.0
22,239606,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3354101,42.0,5.0,37.0
1,12,479151,14.0,2.0,0.0
2,15,239446,79.0,0.0,74.0
3,18,1437485,6.0,0.0,0.0
4,19,2156300,44.0,2.0,0.0
5,20,479086,18.0,28.0,12.0
6,21,718777,3.0,0.0,0.0
7,22,239606,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42.0,5.0,37.0
1,12,14.0,2.0,0.0
2,15,79.0,0.0,74.0
3,18,6.0,0.0,0.0
4,19,44.0,2.0,0.0
5,20,18.0,28.0,12.0
6,21,3.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,12,Released,14.0
2,15,Released,79.0
3,18,Released,6.0
4,19,Released,44.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21,22
Status,,,,,,,,
Completed,37.0,0.0,74.0,0.0,0.0,12.0,0.0,0.0
Executing,5.0,2.0,0.0,0.0,2.0,28.0,0.0,1.0
Released,42.0,14.0,79.0,6.0,44.0,18.0,3.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21,22
0,Completed,37.0,0.0,74.0,0.0,0.0,12.0,0.0,0.0
1,Executing,5.0,2.0,0.0,0.0,2.0,28.0,0.0,1.0
2,Released,42.0,14.0,79.0,6.0,44.0,18.0,3.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21,22
0,Completed,37.0,0.0,74.0,0.0,0.0,12.0,0.0,0.0
1,Executing,5.0,2.0,0.0,0.0,2.0,28.0,0.0,1.0
2,Released,42.0,14.0,79.0,6.0,44.0,18.0,3.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()